# Earthquake data

This data was collected for the Udacity Exploratory Data Analysis course. In this Notebook we explain all the steps taken for merging the data. 

## Sources

The final set uses these data sources. A description of each variable can be found in the code book **(make reference to code book here)**.

1. **Eartquake data:** The data on earthquake was obtained from the Significance Earthquake Database [available here](http://www.ngdc.noaa.gov/nndc/struts/form?t=101650&s=1&d=1). This is  elaborated by the US National Oceanic and Atmospheric Administration. The dataset contains records of significant earthquakes and a series of variables related to their magnitude and damage. The dataset spans from 2150 BC till present. Check code book and page to see operational definition of significant earthquake.
2. **World shape file:** We use the world shape file obtained in this [page](http://thematicmapping.org/downloads/world_borders.php). We use this shapefile for two reasons: first, it makes the process of merging the earthquake dataset more easy. The earthquake data contains a column with the full country name. However, for merging purposes, using the country name has problems, given that the same country may present slight or complete name variations in different data sets. The world shape file has the advantage to count with a ISO3 code for country. This is guaranteed to be a unique identifier per country. Second, with the shape file, we can also create maps to better visualize our results. In order to perform the merge, we use a spatial merge. More explanation to follow.
3. **Penn World Tables:** The Penn World tables is a database that collects economic information from countries. The information runs from 1950, onwards. More information about the dataset is available [here](http://www.rug.nl/research/ggdc/data/pwt/).
4. **Polity IV project:** This is a dataset that is widely used in political science research. It collects information about democratic/autocratic features for countries. The range of the dataset spans from 1800 to nowadays. From this dataset we extract the variable polityIV, which aims to be an score for describing how democratic a country is. Note that in order to merge this variable with our final data set we needed to carefully match the country names. All of the steps are explained below. 

In [674]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

## Working with earthquake data

First, we import the eartquake data. Afterwards, we decide on a certain conditions our dataset should have. For instance, we decide that we are going to concentrate only on earthquakes that happened from 1900 onwards. Also, we want complete our earthquake to have complete registries in terms of year, days and moths in which they occurred. 

In [675]:
#reading data into pandas
earthquake_data = pd.read_csv("results.tsv", sep = "\t", dtype = {"YEAR": str, "MONTH": str, 
                                                                  "DAY": str, "HOUR": str, "MINUTE": str,
                                                                 "SECOND": str})

In [676]:
#dropping incomplete date cases
earthquake_data = earthquake_data.dropna(axis = 0, subset=["YEAR", "MONTH", "DAY"])
#selecting earthaquakes from 1900s onwards
earthquake_data = earthquake_data.loc[earthquake_data.loc[:, "YEAR"].astype(int) >= 1900,  :]
#creating a unique data
earthquake_data["Date"] = earthquake_data[["YEAR", "MONTH", "DAY"]].apply(lambda x: "-".join(x), axis = 1)
earthquake_data["Date"] = pd.to_datetime(earthquake_data["Date"], format="%Y-%m-%d")

In [677]:
#Importing world shape file
world_shp = gpd.GeoDataFrame.from_file('/Users/Jose/Documents/Udacity/data_science/earthquake_project/world_borders/TM_WORLD_BORDERS-0.3.shp')

## Merging earthquake data with world shape file
Next, we prepare our earthquake data for a spatial merge with the world shapefile. The world shapefile contains the geometries of each country in the world. On the other hand, the earthquake data has latitude and longitude coordinates for the earthquakes' location. The spatial join will look if the latitude and longitude fall within the boundaries of a country geometry and, if so, merge it appropriately. 

In order for the merge to yield appropriate results, we need to do the following: first, translate the latitude and longitude coordinates into their appropriate shape form, that is points. Then, we need to take out all the cases where one of these variables is missing. Otherwise, the join wil raise an error.

In [678]:
#Changing type of LAT / LONG to numeric
earthquake_data[["LONGITUDE", "LATITUDE"]] = earthquake_data[["LONGITUDE", "LATITUDE"]].apply(pd.to_numeric, errors='coerce')
#Dropping missing LAT/LONG values
earthquake_data = earthquake_data.dropna(axis = 0, subset=["LONGITUDE", "LATITUDE"])
#Transforming LAT/LONG to points
earthquake_data["Points"] = earthquake_data[["LONGITUDE", "LATITUDE"]].apply(lambda row: Point(row["LONGITUDE"], row["LATITUDE"]), axis=1)

In [679]:
#Passing earthquake data to a GeoPandas dataframe
earthquake_data = gpd.GeoDataFrame(earthquake_data)
#Declaring the coordinate system for the earthquake data 
earthquake_data.crs = world_shp.crs
earthquake_data["geometry"] = earthquake_data["Points"]

In [680]:
#Performing spatial join
joined_data = gpd.sjoin(earthquake_data, world_shp, how="right", op='intersects')
joined_data = joined_data.reset_index()
#Dropping cases with no earthquakes
joined_data.dropna(axis = 0, subset=["LATITUDE", "LONGITUDE"], inplace=True)
#Cleaning data set
joined_data = joined_data.drop(["index_left", "LAT", "LON", "POP2005"], axis=1)

## Using Penn World Tables

We use two variables from the Penn World Tables, GDP (expenditure side) at purchase parity prices and population. Note that this data set only runs from 1950 and it has a lot of missing for most of the countries in early years. As this is a yearly data, we make sure to merge on year and country. Note that the earthquake data gather so far has no fixed time span, as an earthquake could happen at any moment. 

In [712]:
#Reading PENN tables
penn_tables = pd.read_excel("pwt90.xlsx", sheetname="Data")

In [713]:
#We select a subset of the data (rgdpe and pop)
penn_tables = penn_tables.loc[:, ["countrycode", "country", "year", "rgdpe", "pop"]]
#We drop all of the empty registries
penn_tables = penn_tables.dropna()

In [686]:
#Checking data types for merging purposes
penn_tables.dtypes

countrycode     object
country         object
year             int64
rgdpe          float64
pop            float64
dtype: object

In [687]:
#Changing year type for merging purposes (matching of types)
joined_data["YEAR"] = joined_data["YEAR"].astype(int)
#Right merge of both data sets (we want to keep earthquake records 
##even though there is no complete data in Penn World Tables)
earth_penn = pd.merge(penn_tables, joined_data, left_on = ["year", "countrycode"], right_on = ["YEAR", "ISO3"], how = "right", indicator = True )

In [692]:
#Checking merge results for merged data
earth_penn[(earth_penn["_merge"] == "right_only") & (earth_penn["YEAR"] > 1950)] 
#dropping "_merge" variable used to check if merge was appropriately carried out
earth_penn.drop("_merge", axis = 1, inplace=True)

## Using Polity IV data
The polity IV data uses a particular country code identifier. Therefore, we needed to make sure to find the appropriate correspondance between that country code and the ISO3 country code used in our dataset. 

For this purpose, we take the following steps:
1. We import the country codes data sets from the Polity IV data. We also subset our earthquake data and only take the country codes variables. This will make the correspondance process lighter, as we will only be working with the variables we are actually interested in.
2. We use the following strategy: first we merge on country codes of both data sets and then on country names. 
3. **merging on coutry codes **. Problems found: a proper matching case between ISO3 code and PolityIV country code exists, however the mathching is false. For example, while there was a match for SLV in both data sets, this was not appropriate. In ISO3 this corresponds to El Salvador and in PolityIV to Slovenia. We take appropriate steps to check that the corresponding match is found.
4. **merging on names **. Once taken out all of those cases where an appropriate country code matched existed, we use country names to check if there are more matches available. Luckily, all the matches produced here are rightly done so no further correction is needed.
5. **non merged countries**. Inspecting the difference between our current matched countries and the earthquake data set we found that there are still 8 countries for which there is no appropriate match. For this cases we performed a search in the PolityIV country names to see if they contain a patter related to that country. In the end we got to find appropriate matches for 5 countries. For this countries we manually select their corresponding PolityIV identifier and add it to our final correspondance list. The other countries, namely Puerto Rico, Palestine and Serbia are not found in the PolityIV data. This may have to with the working definition of country implied in both datasets.

Having checked that we have the appropriate correspondance for countries between polityIV and our earthquake data we perform the merge to get our final dataset. 

In [696]:
# Reading in country codes
c_codes = pd.read_csv("COW country codes.csv")
c_codes = c_codes.drop_duplicates()

In [697]:
c_codes.columns

Index([u'StateAbb', u'CCode', u'StateNme'], dtype='object')

In [698]:
#reading in country codes for our earthquake dataset
country_checker = earth_penn[["ISO3", "NAME"]].drop_duplicates()

In [699]:
#Merging both datasets on country codes
first_merge = pd.merge(c_codes, country_checker, left_on="StateAbb", right_on="ISO3", how="outer", indicator=True)
#Non correspondance for earthquake data countries finding names in earthquake data
resolving_names_iso3 = country_checker.query('NAME == ["Austria", "Montenegro", "Slovenia"]')
#Finding names in polityIV country data
resolving_names_cow = first_merge.query('StateNme == ["Montenegro", "Slovenia"]')[["StateAbb","StateNme"]]
#Generating our country correspondance dataframe
trans_iso_cow = pd.merge(resolving_names_iso3, resolving_names_cow, left_on ="NAME", right_on="StateNme" )

In [700]:
#Working on non correspondances for earthquake data set
#Finding corresponding names in PolityIV country identifiers
resolving_names_cow =  c_codes[(c_codes.StateNme.str.contains("Salva")) | 
                               (c_codes.StateNme.str.contains("Mongo")) | 
                                (c_codes.StateNme.str.contains("Austral"))][["StateAbb","StateNme"]]
resolving_names_iso3 = country_checker.query('NAME == ["El Salvador", "Mongolia", "Australia"]')
#Adding ready country codes correspondances to previously created dataframe
trans_iso_cow = pd.concat((trans_iso_cow, pd.merge(resolving_names_iso3, 
                                resolving_names_cow, left_on ="NAME", right_on="StateNme")), 
          ignore_index= True)
trans_iso_cow = trans_iso_cow.drop_duplicates()


In [701]:
#Adding cases to our correspondance list for which merging was appropriately performed
both_cases = first_merge[first_merge["_merge"] == "both"]
#Taking out bad cases for which solution was found
good_both_cases = both_cases.loc[~both_cases["ISO3"].isin(trans_iso_cow["ISO3"]),:][["StateAbb","StateNme", "ISO3", "NAME"]]
#Adding to our final correspondance dataframe
trans_iso_cow = pd.concat((trans_iso_cow, good_both_cases), 
          ignore_index= True)

In [702]:
#Merging on names
right_only = first_merge.loc[first_merge["_merge"] == "right_only", ["NAME", "ISO3"]]
right_merge_cc = pd.merge(right_only, c_codes, left_on = "NAME", right_on = "StateNme" )
right_merge_cc = right_merge_cc.loc[:, ["ISO3", "NAME", "StateAbb", "StateNme"]]
#Adding to our final correspondance list after checking appropriate correspondance
trans_iso_cow = pd.concat((trans_iso_cow, right_merge_cc), 
          ignore_index= True)
trans_iso_cow = trans_iso_cow.drop_duplicates()

In [704]:
#Checking which countries in earthquake do not have their appropriate correspondance
lacking_countries = country_checker.loc[~country_checker["ISO3"].isin(trans_iso_cow["ISO3"]), :]

,ISO3,NAME
952,MKD,The former Yugoslav Republic of Macedonia
953,MMR,Burma
1161,SRB,Serbia
1285,TZA,United Republic of Tanzania
1688,PSE,Palestine
1882,LCA,Saint Lucia
2169,LBY,Libyan Arab Jamahiriya
2173,PRI,Puerto Rico


In [705]:
#Finding correct correspondances for those countries in PolityIV country data
pattern = r'Luc|Libya|Tanza|anma|Mac'
finding_matches = c_codes.loc[(c_codes.StateNme.str.contains(pattern)),:]
finding_matches.loc[:,"right_index_match" ] = pd.Series([1882, 952, 1285, 2169, 953], index=finding_matches.index)
concat_matched = pd.merge(finding_matches, lacking_countries, left_on="right_index_match", right_index=True).loc[:, ["ISO3", "NAME", "StateAbb", "StateNme"]]
##Adding manually matched index to final dataset.
trans_iso_cow = pd.concat((trans_iso_cow, concat_matched), 
          ignore_index= True)

In [708]:
country_translate = trans_iso_cow.loc[:, ["ISO3", "StateAbb"]]

In [ ]:
earth_penn = pd.merge(earth_penn, country_translate, on="ISO3", how = "left", indicator=True)

In [725]:
earth_penn = earth_penn.drop(["_merge"], axis=1)

In [727]:
#Reading PolityIV dataset
polityIV = pd.read_excel("p4v2015.xls")

In [738]:
polityIV = polityIV.loc[: ,["scode", "country", "year", "polity2"]]

In [742]:
earthquake_data = pd.merge(earth_penn, polityIV, 
         left_on = ["StateAbb", "YEAR"], 
         right_on = ["scode", "year"], 
         how = "left",
        indicator = True)

In [ ]:
#Clening data_set. Selecting variables we'll work with
drop_columns = ['countrycode', 'country_x', 'year_x', 'I_D', 
 'COUNTRY', 'STATE', 'LOCATION_NAME','REGION_CODE',
'StateAbb', 'scode',  'country_y', 'year_y', '_merge']
earthquake_data = earthquake_data.drop(drop_columns, axis=1)

In [768]:
geom_data = earthquake_data[['geometry', 'ISO3', 'YEAR']]
geom_data.to_csv("geometry_data")

In [771]:
earthquake_data = earthquake_data.drop("geometry", axis=1)

In [772]:
earthquake_data.to_csv("earthquake_data")